In [1]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold, KFold
from sklearn import linear_model, metrics, preprocessing
from math import sqrt, floor
from datetime import datetime, timedelta
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input, LSTM,Dense,RepeatVector, TimeDistributed , LeakyReLU
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, IsolationForest
from sklearn.metrics import r2_score, make_scorer, accuracy_score, mean_squared_error, mean_absolute_error
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
import tensorflow as tf
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
import pandas as pd
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
import time
from sklearn.pipeline import make_pipeline, Pipeline
import math
from sklearn.decomposition import PCA
from numpy import array
from keras.utils.vis_utils import plot_model
import keras
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input,Conv2D, Conv1D, Conv1DTranspose,MaxPooling1D

import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from keras import optimizers, Sequential
from sklearn.metrics import f1_score
import pickle
import ast


path_errors = '/mnt/stud/home/vpulagura/dataset/exp2/errortimestamps.csv'
path_features = '/mnt/stud/home/vpulagura/dataset/exp2/featureslist.csv'
df_errors = pd.read_csv(path_errors)
df_features = pd.read_csv(path_features)
total_features = list(df_features[(df_features['True/False'] == 'TRUE') | (df_features['True/False'] == 'NotAvailable')]['Features'])
tot_feat = pd.read_csv('/mnt/stud/home/vpulagura/experiment/AutoencoderModels/totfeat.csv')
tot_feat = list(tot_feat['feat'])

def create_directory(directory_path):
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

def flatten(X):
    flattened_X = np.empty((X.shape[0], X.shape[2]))  # sample x features array.
    for i in range(X.shape[0]):
        flattened_X[i] = X[i, (X.shape[1]-1), :]
    return(flattened_X)
def temporalize(X, lookback=8):

    output_X = []
    for i in range(len(X) - lookback - 1):
        t = []
        for j in range(1, lookback + 1):
            t.append(X[[(i + j + 1)], :])
        output_X.append(t)
    return np.squeeze(np.array(output_X))


def create_lstm_autoencoder_model(layer1=20, layer2=6, lr=0.01):
    lstm_autoencoder = Sequential()
    lstm_autoencoder.add(LSTM(layer1, activation='relu', input_shape=(8, 137), return_sequences=True))
    lstm_autoencoder.add(LSTM(layer2, activation='relu', return_sequences=False))
    lstm_autoencoder.add(RepeatVector(8))
    lstm_autoencoder.add(LSTM(layer2, activation='relu', return_sequences=True))
    lstm_autoencoder.add(LSTM(layer1, activation='relu', return_sequences=True))
    lstm_autoencoder.add(TimeDistributed(Dense(137)))

    adam = tf.keras.optimizers.Adam(lr)
    lstm_autoencoder.compile(loss='mse', optimizer=adam)    
    return lstm_autoencoder

def getHyperParams(file, df_hyp):
    param_dict = ast.literal_eval(df_hyp.loc[df_hyp['Inverter'] == file]['Feature Importance'].values[0])
    print(param_dict)
    return param_dict['model__batch_size'], param_dict['model__epochs'],param_dict['model__layer1'], param_dict['model__layer2']

callback1 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1)
dayWiseResultsTest = {}
dayWiseResultsTrain = {}
lookback = 8
for itr in range(1,4):
    SMA_RESULTS_PATH_TRAIN = f'/mnt/stud/home/vpulagura/experiment/SMA_2W_1M_3M/Results/Iter{itr}/train/'
    SMA_RESULTS_PATH_TEST = f'/mnt/stud/home/vpulagura/experiment/SMA_2W_1M_3M/Results/Iter{itr}/test/'
    for dayStart in [14, 30, 90]:
        hyp = '2W' if dayStart == 14 else '1M' if dayStart == 30 else '3M' 
        df_hyp = pd.read_csv(f"/mnt/stud/home/vpulagura/experiment/SMA_2W_1M_3M/Hyperparameters/LstmAE{hyp}.csv")
        print(dayStart)
        daysrange = str(dayStart)
        source_dict_results = {}
        source_dict_results_train = {}
        lst=[]
        for i in range(19):
            tot_feat.append('ErrBits')
            print(i)
            inverter = i
            df_inv_0 = pd.read_csv(f'/mnt/stud/home/vpulagura/experiment/AutoencoderModels/downsampledata_PandasMeanNew/{inverter}.csv')
            df_inv_0['Timestamp'] = pd.to_datetime(df_inv_0['Timestamp'])
            df_inv_0.sort_values(by='Timestamp', inplace=True)
            df_inv_0 = df_inv_0.set_index(df_inv_0['Timestamp'])

            format = '%Y-%m-%d %H:%M:%S'
            error_date = datetime.strptime(df_errors[(df_errors['Inverter'] == inverter) & (df_errors['had_failure'] == True)]['failure_time'].values[0][0:19], format)
            start_days = 30+dayStart
            start_date = error_date-timedelta(days= start_days)
            end_date = error_date+timedelta(days= 1*30)
            split_date = error_date-timedelta(days= 1*30)
            df_inv_0 = df_inv_0[(df_inv_0.index >= start_date) & (df_inv_0.index < end_date)]
            df_inv_0 = pd.get_dummies(df_inv_0)

            for feat in tot_feat:
                if feat not in df_inv_0.columns:
                    df_inv_0[feat] = 0
            df_inv_0 = df_inv_0[tot_feat]
            df_inv_0 = df_inv_0.dropna()

            df_target = df_inv_0[['ErrBits']]
            tot_feat.remove('ErrBits')
            df_inv_0 = df_inv_0[tot_feat]

            Xtrain_0, Xtest_0 = df_inv_0[(df_inv_0.index >= start_date) & (df_inv_0.index< split_date)], df_inv_0[(df_inv_0.index >= split_date) & (df_inv_0.index< end_date)]
            print(error_date, start_date, split_date, end_date)
            print(df_inv_0.shape, Xtrain_0.shape, Xtest_0.shape)
            Timestamp_train_0, Timestamp_test_0 = df_inv_0[(df_inv_0.index >= start_date) & (df_inv_0.index< split_date)].index, df_inv_0[(df_inv_0.index >= split_date) & (df_inv_0.index< end_date)].index
            ytrain, ytest = df_target[(df_target.index >= start_date) & (df_target.index< split_date)], df_target[(df_target.index >= split_date) & (df_target.index< end_date)]
            ytrain, ytest = ytrain[lookback+1:], ytest[lookback+1:]
            print(Xtrain_0.shape)
            if Xtrain_0.shape[0]>1:
                X_scaler = MinMaxScaler()
                X_scaler.fit(Xtrain_0)
                xtrain = X_scaler.transform(Xtrain_0)
                xtest = X_scaler.transform(Xtest_0)
                xtrain = temporalize(xtrain)
                xtest = temporalize(xtest)
                model__batch_size, model__epochs, model__layer1, model__layer2 = getHyperParams(inverter, df_hyp)
                model  = create_lstm_autoencoder_model(layer1=model__layer1, layer2=model__layer2)
                model.fit(xtrain, xtrain, epochs=model__epochs, batch_size=model__batch_size, verbose=False,validation_split=0.1, callbacks=callback1)
                test_enc = model.predict(xtest)
                train_enc = model.predict(xtrain)
                test_mses = np.mean(np.power(flatten(xtest)- flatten(test_enc), 2), axis=1)
                train_mses = np.mean(np.power(flatten(xtrain)- flatten(train_enc), 2), axis=1)
                test_maes = np.mean(flatten(xtest)- flatten(test_enc), axis=1)
                train_maes = np.mean(flatten(xtrain)- flatten(train_enc), axis=1)

                mse_threshold = np.max(train_mses)
                mae_threshold = np.max(train_maes)

                ypred_mse = np.where(test_mses>mse_threshold,1,0)
                ypred_mae = np.where(test_maes>mae_threshold,1,0)


                test_df = pd.DataFrame(ytest)
                test_df['mse_scores'] = test_mses
                test_df['ytrue'] = np.where(test_df['ErrBits']>0, 1, 0)
                test_df['mae_scores'] = test_maes
                test_df['ypred_mse'] = ypred_mse
                test_df['ypred_mae'] = ypred_mae
                lst.append(f1_score(y_true=test_df['ytrue'], y_pred=test_df['ypred_mse']))
                test_df = test_df[['ytrue','ypred_mse','ypred_mae','mse_scores', 'mae_scores']]
                source_dict_results[f'inv_{i}'] = test_df
                
                train_df = pd.DataFrame(ytrain)
                train_df['mse_scores'] = train_mses
                train_df['ytrue'] = np.where(train_df['ErrBits']>0, 1, 0)
                train_df['mae_scores'] = train_maes
                train_df = train_df[['ytrue','mse_scores', 'mae_scores']]
                source_dict_results_train[f'inv_{i}'] = train_df
                
        dayWiseResultsTest[dayStart] = source_dict_results 
        dayWiseResultsTrain[dayStart] = source_dict_results_train
        print(lst)
    with open(f'{SMA_RESULTS_PATH_TRAIN}/train_STL_LstmAE.pkl', 'wb') as f:
        pickle.dump(dayWiseResultsTrain, f) 
    with open(f'{SMA_RESULTS_PATH_TEST}/test_STL_LstmAE.pkl', 'wb') as f:
        pickle.dump(dayWiseResultsTest, f) 

2023-08-31 08:17:59.520698: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-31 08:17:59.520753: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


14
0
2019-10-11 08:24:20 2019-08-28 08:24:20 2019-09-11 08:24:20 2019-11-10 08:24:20
(14406, 137) (2761, 137) (11645, 137)
(2761, 137)
{'model__batch_size': 128, 'model__epochs': 100, 'model__layer1': 64, 'model__layer2': 64}


2023-08-31 08:19:38.467749: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-31 08:19:38.467852: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-08-31 08:19:38.467909: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-08-31 08:19:38.467964: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2023-08-31 08:19:38.498077: W tensorflow/stream_executor/platform/default/dso_loader.cc:64

Epoch 00098: early stopping
1
2019-04-02 15:20:48 2019-02-17 15:20:48 2019-03-03 15:20:48 2019-05-02 15:20:48
(14115, 137) (2884, 137) (11231, 137)
(2884, 137)
{'model__batch_size': 128, 'model__epochs': 200, 'model__layer1': 64, 'model__layer2': 32}
Epoch 00061: early stopping
2
2018-12-28 09:46:38 2018-11-14 09:46:38 2018-11-28 09:46:38 2019-01-27 09:46:38
(14142, 137) (2884, 137) (11258, 137)
(2884, 137)
{'model__batch_size': 256, 'model__epochs': 200, 'model__layer1': 128, 'model__layer2': 64}
Epoch 00177: early stopping
3
2018-10-13 16:26:39 2018-08-30 16:26:39 2018-09-13 16:26:39 2018-11-12 16:26:39
(13338, 137) (2199, 137) (11139, 137)
(2199, 137)
{'model__batch_size': 128, 'model__epochs': 200, 'model__layer1': 128, 'model__layer2': 64}
Epoch 00108: early stopping
4
2018-10-13 13:18:07 2018-08-30 13:18:07 2018-09-13 13:18:07 2018-11-12 13:18:07
(13150, 137) (2225, 137) (10925, 137)
(2225, 137)
{'model__batch_size': 256, 'model__epochs': 200, 'model__layer1': 128, 'model__layer2

{'model__batch_size': 128, 'model__epochs': 100, 'model__layer1': 64, 'model__layer2': 64}
14
2019-10-12 15:00:20 2019-08-13 15:00:20 2019-09-12 15:00:20 2019-11-11 15:00:20
(18498, 137) (6159, 137) (12339, 137)
(6159, 137)
{'model__batch_size': 128, 'model__epochs': 200, 'model__layer1': 64, 'model__layer2': 64}
Epoch 00091: early stopping
15
2019-10-13 09:51:49 2019-08-14 09:51:49 2019-09-13 09:51:49 2019-11-12 09:51:49
(18495, 137) (6162, 137) (12333, 137)
(6162, 137)
{'model__batch_size': 128, 'model__epochs': 200, 'model__layer1': 64, 'model__layer2': 64}
Epoch 00156: early stopping
16
2019-10-26 08:15:14 2019-08-27 08:15:14 2019-09-26 08:15:14 2019-11-25 08:15:14
(18520, 137) (6180, 137) (12340, 137)
(6180, 137)
{'model__batch_size': 128, 'model__epochs': 200, 'model__layer1': 64, 'model__layer2': 64}
17
2019-10-18 07:26:00 2019-08-19 07:26:00 2019-09-18 07:26:00 2019-11-17 07:26:00
(18519, 137) (6180, 137) (12339, 137)
(6180, 137)
{'model__batch_size': 128, 'model__epochs': 100,

Epoch 00167: early stopping
5
2019-05-19 09:01:02 2019-04-05 09:01:02 2019-04-19 09:01:02 2019-06-18 09:01:02
(15044, 137) (2763, 137) (12281, 137)
(2763, 137)
{'model__batch_size': 128, 'model__epochs': 200, 'model__layer1': 64, 'model__layer2': 64}
Epoch 00153: early stopping
6
2018-06-12 10:55:50 2018-04-29 10:55:50 2018-05-13 10:55:50 2018-07-12 10:55:50
(13520, 137) (2655, 137) (10865, 137)
(2655, 137)
{'model__batch_size': 256, 'model__epochs': 200, 'model__layer1': 128, 'model__layer2': 64}
Epoch 00101: early stopping
7
2019-05-27 04:57:13 2019-04-13 04:57:13 2019-04-27 04:57:13 2019-06-26 04:57:13
(15203, 137) (2884, 137) (12319, 137)
(2884, 137)
{'model__batch_size': 256, 'model__epochs': 200, 'model__layer1': 128, 'model__layer2': 64}
Epoch 00103: early stopping
8
2019-09-21 15:53:48 2019-08-08 15:53:48 2019-08-22 15:53:48 2019-10-21 15:53:48
(11485, 137) (2884, 137) (8601, 137)
(2884, 137)
{'model__batch_size': 128, 'model__epochs': 200, 'model__layer1': 64, 'model__layer2':

KeyboardInterrupt: 